In [1]:
%matplotlib inline
import pandas as pd
import librosa as lib
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Audio
import scipy

In [2]:
#Import music dataframe
musicr = pd.read_pickle('DFs/musicr.pkl')

In [3]:
#import motion dataframe 
motion_all = pd.read_pickle('DFs/motion_all.pkl')

In [4]:
#import demographic information dataframe, includes all participants, but we will only use particpant 1 info for now
demos = pd.read_pickle('DFs/demos_EDA.pkl')

In [5]:
demos.loc[demos['Sex'].isnull(), 'Sex']=1

Reminder of what each look like

In [6]:
musicr.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,A_sharp,B
0,0.391866,0.271475,0.543743,0.414470,0.615711,0.905505,0.693868,1.0,0.873349,0.428765,0.196232,0.178688
1,0.391006,0.270759,0.542460,0.419167,0.611465,0.895052,0.696120,1.0,0.865096,0.424725,0.202718,0.177431
2,0.384290,0.269616,0.530845,0.424728,0.596871,0.879191,0.701280,1.0,0.854621,0.410783,0.205452,0.178596
3,0.368410,0.266402,0.511752,0.433351,0.582881,0.878751,0.706602,1.0,0.850467,0.397394,0.211017,0.174648
4,0.356740,0.260931,0.492235,0.435323,0.570510,0.877007,0.707534,1.0,0.849687,0.381047,0.214705,0.177209


In [7]:
motion_all.head()

,index,X,Y,Z,PID,X_disp_start,Y_disp_start,Z_disp_start,X_scale_mean,Y_scale_mean,...,step_eucl_5s_min,step_eucl_5s_max,step_eucl_10s_avg,step_eucl_10s_min,step_eucl_10s_max,step_eucl_20s_avg,step_eucl_20s_min,step_eucl_20s_max,millisecond,segment
0,1900-01-01 00:00:00.000,159.6975,1844.71755,1732.09535,1,0.0000,0.00000,0.00000,-0.776898,-15.612761,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,silence
1,1900-01-01 00:00:00.200,160.2117,1844.63625,1732.06770,1,0.5142,-0.08130,-0.02765,-0.262698,-15.694061,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.0,silence
2,1900-01-01 00:00:00.400,160.5355,1846.72830,1732.12305,1,0.8380,2.01075,0.02770,0.061102,-13.602011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,400.0,silence
3,1900-01-01 00:00:00.600,160.6253,1849.30315,1732.22560,1,0.9278,4.58560,0.13025,0.150902,-11.027161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,600.0,silence
4,1900-01-01 00:00:00.800,160.5194,1850.78495,1732.32555,1,0.8219,6.06740,0.23020,0.045002,-9.545361,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,800.0,silence


In [8]:
motion_all = motion_all.sort_values(by='index').bfill().ffill()
motion_all

,index,X,Y,Z,PID,X_disp_start,Y_disp_start,Z_disp_start,X_scale_mean,Y_scale_mean,...,step_eucl_5s_min,step_eucl_5s_max,step_eucl_10s_avg,step_eucl_10s_min,step_eucl_10s_max,step_eucl_20s_avg,step_eucl_20s_min,step_eucl_20s_max,millisecond,segment
0,1900-01-01 00:00:00,159.69750,1844.71755,1732.09535,1,0.00000,0.00000,0.00000,-0.776898,-15.612761,...,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.0,silence
0,1900-01-01 00:00:00,862.99300,958.49190,1796.26815,14,0.00000,0.00000,0.00000,-6.455397,-4.057275,...,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.0,silence
0,1900-01-01 00:00:00,33.82225,194.65595,1716.48845,72,0.00000,0.00000,0.00000,7.771688,-55.884076,...,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.0,silence
0,1900-01-01 00:00:00,137.61695,1747.14260,1732.19940,66,0.00000,0.00000,0.00000,-5.488903,0.167195,...,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.0,silence
0,1900-01-01 00:00:00,1756.35740,1818.89230,1621.59845,20,0.00000,0.00000,0.00000,0.812413,-7.911999,...,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.0,silence
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1780,1900-01-01 00:05:56,886.40600,1745.68500,1703.42800,31,-15.39380,7.12465,-2.57050,-5.991272,11.679101,...,-2.017242,2.040888,0.013868,-2.017242,2.040888,0.003825,-2.017242,2.040888,356000.0,edm
1780,1900-01-01 00:05:56,2546.79000,1059.63200,1843.47000,46,-6.35290,9.82690,-1.24235,-3.805608,-7.376386,...,-0.876127,1.104446,0.000650,-1.342707,1.315240,-0.004305,-1.342707,1.315240,356000.0,edm
1780,1900-01-01 00:05:56,826.16500,212.75600,1546.57000,38,-18.45770,15.96590,-0.28060,-5.079797,14.532276,...,-1.763284,1.353903,-0.000212,-2.839181,2.776722,-0.001598,-2.839181,2.776722,356000.0,edm
1780,1900-01-01 00:05:56,1730.29000,971.26100,1768.06800,45,-3.36550,48.78515,-10.93705,-8.864376,6.185460,...,-1.012576,0.875311,-0.003141,-1.210867,1.012252,0.001353,-1.382512,1.334182,356000.0,edm


In [9]:
demos.head()

,PID,SID,Age,Sex,Height,Listen,Produce,Dance,Exercise,Tiresome,age_bin,height_bin
0,1,A1,23,0.0,1.72,5,30.0,1.0,7.0,3,3,2
1,2,A2,24,0.0,1.67,10,10.0,2.0,10.0,1,3,2
2,3,A3,27,1.0,1.63,14,1.0,4.0,3.0,4,4,1
3,4,A4,27,0.0,1.75,5,20.0,2.0,2.0,4,4,3
4,5,A9,24,1.0,1.64,15,0.0,2.0,6.0,2,3,1


### Matching Times Between DataFrames
We need to match the musics and motion DataFrames together. We will create a 'time' column that represents milliseconds as we already have for the motion data. The music starts at 3 minutes or 180000 milliseconds.

In [10]:
musicr['millisecond'] = list(range(180000,356010, 10))

In [11]:
musicr.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,A_sharp,B,millisecond
0,0.391866,0.271475,0.543743,0.414470,0.615711,0.905505,0.693868,1.0,0.873349,0.428765,0.196232,0.178688,180000
1,0.391006,0.270759,0.542460,0.419167,0.611465,0.895052,0.696120,1.0,0.865096,0.424725,0.202718,0.177431,180010
2,0.384290,0.269616,0.530845,0.424728,0.596871,0.879191,0.701280,1.0,0.854621,0.410783,0.205452,0.178596,180020
3,0.368410,0.266402,0.511752,0.433351,0.582881,0.878751,0.706602,1.0,0.850467,0.397394,0.211017,0.174648,180030
4,0.356740,0.260931,0.492235,0.435323,0.570510,0.877007,0.707534,1.0,0.849687,0.381047,0.214705,0.177209,180040


# Feature Creation
We are going to create some additonal features from the note amplitudes including rolling averages, mins, maxs and rates of change

In [12]:
def note_features(note_column_df, note):
    feature_df = pd.DataFrame()
    feature_df[note + '_5sec_avg'] = note_column_df.rolling(500).mean()
    feature_df[note + '_2.5sec_avg'] = note_column_df.rolling(250).mean()
    feature_df[note + '_1sec_avg'] = note_column_df.rolling(100).mean()
    feature_df[note + '_10sec_max'] = note_column_df.rolling(500).max()
    feature_df[note + '_5sec_max'] = note_column_df.rolling(250).max()
    feature_df[note + '_1sec_max'] = note_column_df.rolling(100).max()
    feature_df[note + '_10sec_min'] = note_column_df.rolling(500).min()
    feature_df[note + '_5sec_min'] = note_column_df.rolling(250).min()
    feature_df[note + '_1sec_min'] = note_column_df.rolling(100).min()
    feature_df[note + '_10sec_diff'] = note_column_df.rolling(500).max() - note_column_df.rolling(250).min()
    feature_df[note + '_5sec_diff'] = note_column_df.rolling(250).max() - note_column_df.rolling(50).min()
    feature_df[note + '_1sec_diff'] = note_column_df.rolling(100).max() - note_column_df.rolling(100).min()
    return feature_df

In [13]:
musicr

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,A_sharp,B,millisecond
0,0.391866,0.271475,0.543743,0.414470,0.615711,0.905505,0.693868,1.000000,0.873349,0.428765,0.196232,0.178688,180000
1,0.391006,0.270759,0.542460,0.419167,0.611465,0.895052,0.696120,1.000000,0.865096,0.424725,0.202718,0.177431,180010
2,0.384290,0.269616,0.530845,0.424728,0.596871,0.879191,0.701280,1.000000,0.854621,0.410783,0.205452,0.178596,180020
3,0.368410,0.266402,0.511752,0.433351,0.582881,0.878751,0.706602,1.000000,0.850467,0.397394,0.211017,0.174648,180030
4,0.356740,0.260931,0.492235,0.435323,0.570510,0.877007,0.707534,1.000000,0.849687,0.381047,0.214705,0.177209,180040
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17596,0.551258,0.239016,0.590822,0.848327,0.556109,0.694828,1.000000,0.846755,0.592610,0.795401,0.520809,0.609293,355960
17597,0.486828,0.255034,0.616595,0.832311,0.567620,0.685870,1.000000,0.849619,0.508727,0.755731,0.539001,0.600682,355970
17598,0.418250,0.280371,0.653649,0.814706,0.601733,0.692381,1.000000,0.866687,0.420425,0.728755,0.549097,0.572606,355980
17599,0.418250,0.280371,0.653649,0.814706,0.601733,0.692381,1.000000,0.866687,0.420425,0.728755,0.549097,0.572606,355990


In [14]:
for note in musicr.columns:
    if note != 'millisecond':
        musicr = musicr.merge(note_features(musicr[note], note), left_index=True, right_index=True)

In [15]:
musicr.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,B_1sec_avg,B_10sec_max,B_5sec_max,B_1sec_max,B_10sec_min,B_5sec_min,B_1sec_min,B_10sec_diff,B_5sec_diff,B_1sec_diff
0,0.391866,0.271475,0.543743,0.414470,0.615711,0.905505,0.693868,1.0,0.873349,0.428765,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.391006,0.270759,0.542460,0.419167,0.611465,0.895052,0.696120,1.0,0.865096,0.424725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.384290,0.269616,0.530845,0.424728,0.596871,0.879191,0.701280,1.0,0.854621,0.410783,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.368410,0.266402,0.511752,0.433351,0.582881,0.878751,0.706602,1.0,0.850467,0.397394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.356740,0.260931,0.492235,0.435323,0.570510,0.877007,0.707534,1.0,0.849687,0.381047,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
musicr.fillna(0, inplace=True)
musicr.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,B_1sec_avg,B_10sec_max,B_5sec_max,B_1sec_max,B_10sec_min,B_5sec_min,B_1sec_min,B_10sec_diff,B_5sec_diff,B_1sec_diff
0,0.391866,0.271475,0.543743,0.414470,0.615711,0.905505,0.693868,1.0,0.873349,0.428765,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.391006,0.270759,0.542460,0.419167,0.611465,0.895052,0.696120,1.0,0.865096,0.424725,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.384290,0.269616,0.530845,0.424728,0.596871,0.879191,0.701280,1.0,0.854621,0.410783,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.368410,0.266402,0.511752,0.433351,0.582881,0.878751,0.706602,1.0,0.850467,0.397394,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.356740,0.260931,0.492235,0.435323,0.570510,0.877007,0.707534,1.0,0.849687,0.381047,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#merge the motion and music data together on time column
merged = musicr.merge(motion_all, on='millisecond', how='right')
merged.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,step_eucl_5s_avg,step_eucl_5s_min,step_eucl_5s_max,step_eucl_10s_avg,step_eucl_10s_min,step_eucl_10s_max,step_eucl_20s_avg,step_eucl_20s_min,step_eucl_20s_max,segment
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,silence
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,silence
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,silence
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,silence
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,silence


In [18]:
merged.fillna(0, inplace=True)
merged.head()

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,step_eucl_5s_avg,step_eucl_5s_min,step_eucl_5s_max,step_eucl_10s_avg,step_eucl_10s_min,step_eucl_10s_max,step_eucl_20s_avg,step_eucl_20s_min,step_eucl_20s_max,segment
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,silence
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,silence
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,silence
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,silence
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,0.276899,-0.155883,1.232914,silence


In [19]:
merged = merged.merge(demos, on='PID', how='left')
merged

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,Age,Sex,Height,Listen,Produce,Dance,Exercise,Tiresome,age_bin,height_bin
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,23,0.0,1.72,5,30.0,1.0,7.0,3,3,2
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,23,1.0,1.80,20,30.0,2.0,5.0,3,3,4
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,65,1.0,1.73,4,0.0,2.0,2.0,2,5,3
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,57,0.0,1.79,10,0.0,5.0,5.0,1,5,4
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,27,1.0,1.72,30,4.0,1.0,2.0,3,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130008,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,17,1.0,1.64,15,0.0,2.0,16.0,3,1,1
130009,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,32,0.0,1.84,10,0.0,0.0,3.0,3,5,5
130010,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,20,1.0,1.55,30,20.0,4.0,3.0,3,2,1
130011,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,34,0.0,1.77,15,30.0,0.0,1.0,3,5,4


In [20]:
merged_sort = merged.set_index('index').sort_index()
merged_sort

,C,C_sharp,D,D_sharp,E,F,F_sharp,G,G_sharp,A,...,Age,Sex,Height,Listen,Produce,Dance,Exercise,Tiresome,age_bin,height_bin
index,,,,,,,,,,,,,,,,,,,,,
1900-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,23,0.0,1.72,5,30.0,1.0,7.0,3,3,2
1900-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,23,1.0,1.80,20,30.0,2.0,5.0,3,3,4
1900-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,65,1.0,1.73,4,0.0,2.0,2.0,2,5,3
1900-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,57,0.0,1.79,10,0.0,5.0,5.0,1,5,4
1900-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,27,1.0,1.72,30,4.0,1.0,2.0,3,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1900-01-01 00:05:56,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,17,1.0,1.64,15,0.0,2.0,16.0,3,1,1
1900-01-01 00:05:56,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,32,0.0,1.84,10,0.0,0.0,3.0,3,5,5
1900-01-01 00:05:56,0.370047,0.297515,0.677903,0.787486,0.621999,0.695762,1.0,0.880282,0.364317,0.719848,...,20,1.0,1.55,30,20.0,4.0,3.0,3,2,1


In [21]:
merged_sort.isnull().sum().sort_values(ascending=False).head(10)

C                    0
Z_disp               0
A_sharp_5sec_diff    0
A_sharp_1sec_diff    0
B_5sec_avg           0
B_2.5sec_avg         0
B_1sec_avg           0
B_10sec_max          0
B_5sec_max           0
B_1sec_max           0
dtype: int64

In [22]:
merged_sort.to_pickle('DFs/model_data.pkl')